In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# 1. Load dataset MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalisasi data (0-255 → 0-1)
X_train = X_train / 255.0
X_test = X_test / 255.0

# One-hot encoding label
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 2. Bangun model JST
model = Sequential([
    Flatten(input_shape=(28, 28)),             # Ubah gambar 28x28 → vektor 784
    Dense(128, activation='relu'),             # Hidden layer 1
    Dense(64, activation='relu'),              # Hidden layer 2
    Dense(10, activation='softmax')            # Output layer (10 kelas)
])

# 3. Kompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 4. Latih model
model.fit(X_train, y_train, epochs=5, batch_size=32)

# 5. Evaluasi model
loss, acc = model.evaluate(X_test, y_test)
print(f"Akurasi pada data uji: {acc:.4f}")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8793 - loss: 0.4249
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9673 - loss: 0.1060
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9783 - loss: 0.0695
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9834 - loss: 0.0511
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9875 - loss: 0.0381
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9703 - loss: 0.0964
Akurasi pada data uji: 0.9741


- Coba dengan beberapa parameter lain:
1. Ubah jumlah neuron di hidden layer (misal: 256 dan 128).
2. Tambahkan satu hidden layer lagi.
3. Bandingkan akurasi dan waktu pelatihan.
4. Eksperimen dengan fungsi aktivasi Sigmoid vs ReLU.



In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import time

# Load MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

def run_experiment(h1, h2, h3=None, activation='relu'):
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))

    model.add(Dense(h1, activation=activation))
    model.add(Dense(h2, activation=activation))

    if h3:
        model.add(Dense(h3, activation=activation))

    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    callback = tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=1,
        restore_best_weights=True
    )

    # Kurangi epoch + atur batch
    start = time.time()
    model.fit(X_train, y_train, epochs=5, batch_size=128, callbacks=[callback], verbose=0)
    duration = time.time() - start

    loss, acc = model.evaluate(X_test, y_test, verbose=0)

    print(f"Neuron {h1}-{h2}-{h3}, activation={activation}")
    print(f"Akurasi: {acc:.4f}, Waktu: {duration:.2f} detik\n")

    return acc, duration

acc1, t1 = run_experiment(128, 64, activation='relu')

acc2, t2 = run_experiment(256, 128, activation='relu')

acc3, t3 = run_experiment(256, 128, 64, activation='relu')

acc4, t4 = run_experiment(128, 64, 32, activation='sigmoid')

print("\n=== HASIL PERBANDINGAN EKSPERIMEN ===")
print(f"1. ReLU (128-64)                → Acc: {acc1:.4f}, Time: {t1:.2f} s")
print(f"2. ReLU (256-128)               → Acc: {acc2:.4f}, Time: {t2:.2f} s")
print(f"3. ReLU (256-128-64)            → Acc: {acc3:.4f}, Time: {t3:.2f} s")
print(f"4. Sigmoid (128-64-32)          → Acc: {acc4:.4f}, Time: {t4:.2f} s")


Neuron 128-64-None, activation=relu
Akurasi: 0.9746, Waktu: 13.99 detik

Neuron 256-128-None, activation=relu
Akurasi: 0.9796, Waktu: 20.07 detik

Neuron 256-128-64, activation=relu
Akurasi: 0.9773, Waktu: 21.57 detik

Neuron 128-64-32, activation=sigmoid
Akurasi: 0.9615, Waktu: 19.39 detik


=== HASIL PERBANDINGAN EKSPERIMEN ===
1. ReLU (128-64)                → Acc: 0.9746, Time: 13.99 s
2. ReLU (256-128)               → Acc: 0.9796, Time: 20.07 s
3. ReLU (256-128-64)            → Acc: 0.9773, Time: 21.57 s
4. Sigmoid (128-64-32)          → Acc: 0.9615, Time: 19.39 s
